In [1]:
import pandas as pd


In [2]:
cfs = pd.read_excel("impact_world_plus_2.0.1_dev.xlsx", sheet_name="Sheet1")
cfs = cfs.loc[
    (cfs["Native geographical resolution scale"] == "Country")
    | (cfs["Native geographical resolution scale"] == "Continent")
     ]
cfs.head()

,Unnamed: 0,Impact category,CF unit,Compartment,Sub-compartment,Elem flow name,CAS number,CF value,Elem flow unit,MP or Damage,Native geographical resolution scale
6518,6518,Freshwater acidification,kg SO2 eq,Air,(unspecified),"Ammonia, AD",007664-41-7,0.180141,kg,Midpoint,Country
6519,6519,Freshwater acidification,kg SO2 eq,Air,high. pop.,"Ammonia, AD",007664-41-7,0.180141,kg,Midpoint,Country
6520,6520,Freshwater acidification,kg SO2 eq,Air,low. pop.,"Ammonia, AD",007664-41-7,0.180141,kg,Midpoint,Country
6521,6521,Freshwater acidification,kg SO2 eq,Air,stratosphere + troposphere,"Ammonia, AD",007664-41-7,0.180141,kg,Midpoint,Country
6522,6522,Freshwater acidification,kg SO2 eq,Air,indoor,"Ammonia, AD",007664-41-7,0.180141,kg,Midpoint,Country


In [3]:
# create a column called `location` where we extract teh location from the `Elem flow name` column
# location are capital letters after the last comma
cfs["location"] = cfs["Elem flow name"].str.extract(r',\s*([A-Z]+)$')
cfs["location"].unique()

array(['AD', 'AE', 'AF', 'AG', 'AI', 'AL', 'AM', 'AO', 'AQ', 'AR', 'AS',
       'AT', 'AU', 'AW', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH',
       'BI', 'BJ', 'BM', 'BN', 'BO', 'BR', 'BS', 'BT', 'BV', 'BW', 'BY',
       'BZ', 'CA', 'CC', 'CD', 'CF', 'CG', 'CH', 'CI', 'CK', 'CL', 'CM',
       'CN', 'CO', 'CR', 'CU', 'CV', 'CX', 'CY', 'CZ', 'DE', 'DJ', 'DK',
       'DO', 'DZ', 'EC', 'EE', 'EG', 'EH', 'ER', 'ES', 'ET', 'FI', 'FJ',
       'FK', 'FM', 'FO', 'FR', 'GA', 'GB', 'GD', 'GE', 'GF', 'GG', 'GH',
       'GI', 'GL', 'GM', 'GN', 'GP', 'GQ', 'GR', 'GS', 'GT', 'GU', 'GW',
       'GY', nan, 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IM', 'IN',
       'IO', 'IQ', 'IR', 'IS', 'IT', 'JE', 'JM', 'JO', 'JP', 'KE', 'KG',
       'KH', 'KI', 'KM', 'KN', 'KP', 'KR', 'KW', 'KZ', 'LA', 'LB', 'LC',
       'LI', 'LK', 'LR', 'LS', 'LT', 'LU', 'LV', 'LY', 'MA', 'MC', 'MD',
       'ME', 'MG', 'MH', 'MK', 'ML', 'MM', 'MN', 'MQ', 'MR', 'MS', 'MT',
       'MU', 'MV', 'MW', 'MX', 'MY', 'MZ', 'NA', 'NC

In [4]:
# fix values under `Elem flow name` column, by removing the location and the comma preceding it
cfs["Elem flow name"] = cfs["Elem flow name"].str.rsplit(', ', n=1).str[0]
cfs["Elem flow name"].unique()

array(['Ammonia', 'Ammonia, as N', 'Ammonium carbonate',
       'Ammonium nitrate', 'Ammonium, ion', 'Nitrate', 'Nitric acid',
       'Nitric oxide', 'Nitrite', 'Nitrogen dioxide', 'Nitrogen oxides',
       'Sulfate', 'Sulfur dioxide', 'Sulfur trioxide', 'Sulfuric acid',
       'BOD5, Biological Oxygen Demand', 'COD, Chemical Oxygen Demand',
       'Phosphate', 'Phosphoric acid',
       'Phosphorus compounds, unspecified', 'Phosphorus pentoxide',
       'Phosphorus', 'Phosphorus, total',
       'Occupation, agriculture, mosaic (agroforestry)',
       'Occupation, annual crops', 'Occupation, artificial areas',
       'Occupation, forest, used',
       'Occupation, forest/grassland, not used',
       'Occupation, pasture/meadow', 'Occupation, permanent crops',
       'Occupation, secondary vegetation', 'Occupation, unspecified',
       'Transformation, from agriculture, mosaic (agroforestry)',
       'Transformation, from annual crops',
       'Transformation, from artificial areas',
   

In [5]:
# Load IW <-> ecoinvent mapping
ei_map = pd.read_excel("ei_iw_mapping.xlsx")

In [6]:
# Define the order of versions from highest to lowest priority
version_order = ["3.10", "3.9", "3.8", "3.7.1", "3.7", "3.6", "3.5 or earlier"]

# Convert the column to categorical with the defined order
ei_map["introduced in ei v."] = pd.Categorical(ei_map["introduced in ei v."], categories=version_order, ordered=True)

# Sort the dataframe so that the highest version appears first
ei_map = ei_map.sort_values(by="introduced in ei v.", ascending=False)

# Create the dictionary by preserving the first encountered mapping
mapping_dict = {}
for _, row in ei_map.iterrows():
    is_name = row["iw name"]
    ecoinvent_name = row["ecoinvent name"]
    if is_name not in mapping_dict:  # Only add if it's not already mapped
        mapping_dict[is_name] = ecoinvent_name

In [7]:
mapping_dict

{'1,3-Dioxolan-2-one': '1,3-Dioxolan-2-one',
 'Phthalate, butyl-benzyl-': 'Phthalate, butyl-benzyl-',
 'Pirimicarb': 'Pirimicarb',
 'Piperonyl butoxide': 'Piperonyl butoxide',
 'Pinoxaden': 'Pinoxaden',
 'Picoxystrobin': 'Picoxystrobin',
 'Picloram': 'Picloram',
 'Phthalate, dioctyl-': 'Phthalate, dioctyl-',
 'Phthalate, dimethyl-': 'Phthalate, dimethyl-',
 'Phthalate, dibutyl-': 'Phthalate, dibutyl-',
 'Phosphorus': 'Phosphorus, 18% in apatite, 4% in crude ore, in ground',
 'Benzene, 1-methyl-2-nitro-': 'o-Nitrotoluene',
 nan: 'Phosphorus trichloride',
 'Phosphoric acid': 'Phosphoric acid',
 'Phosphate': 'Phosphate',
 'Pirimiphos methyl': 'Pirimiphos methyl',
 'Platinum': 'Platinum',
 'Profenofos': 'Profenofos',
 'Procymidone': 'Procymidone',
 'Prochloraz': 'Prochloraz',
 'Praseodymium': 'Praseodymium, 0.42% in bastnasite, 0.042% in crude ore, in ground',
 'Fatty acids, c9-13-neo-': 'Potassium soap',
 'Potassium chloride': 'Potassium chloride',
 'Potassium': 'Potassium',
 'Polychlorin

In [8]:
# create a column called `ecoinvent_name` where we map the `Elem flow name` to ecoinvent names
cfs["ecoinvent_name"] = cfs["Elem flow name"].map(mapping_dict)

In [9]:
cfs.head()

,Unnamed: 0,Impact category,CF unit,Compartment,Sub-compartment,Elem flow name,CAS number,CF value,Elem flow unit,MP or Damage,Native geographical resolution scale,location,ecoinvent_name
6518,6518,Freshwater acidification,kg SO2 eq,Air,(unspecified),Ammonia,007664-41-7,0.180141,kg,Midpoint,Country,AD,Ammonia
6519,6519,Freshwater acidification,kg SO2 eq,Air,high. pop.,Ammonia,007664-41-7,0.180141,kg,Midpoint,Country,AD,Ammonia
6520,6520,Freshwater acidification,kg SO2 eq,Air,low. pop.,Ammonia,007664-41-7,0.180141,kg,Midpoint,Country,AD,Ammonia
6521,6521,Freshwater acidification,kg SO2 eq,Air,stratosphere + troposphere,Ammonia,007664-41-7,0.180141,kg,Midpoint,Country,AD,Ammonia
6522,6522,Freshwater acidification,kg SO2 eq,Air,indoor,Ammonia,007664-41-7,0.180141,kg,Midpoint,Country,AD,Ammonia


In [10]:
# we drop unmapped flows
print(cfs.loc[cfs["ecoinvent_name"].isna(), "Elem flow name"].unique())
cfs = cfs.dropna(subset=["ecoinvent_name"])

['Ammonia, as N' 'Ammonium nitrate' 'BOD5, Biological Oxygen Demand'
 'COD, Chemical Oxygen Demand' 'Phosphorus compounds, unspecified'
 'Phosphorus pentoxide' 'Phosphorus, total' 'Kjeldahl-N' 'Nitrogen, total'
 'Water, agri' 'Water, non-agri' 'Water, shallow well, in ground']


In [11]:
# create a `ecoinvent main compartment` column, and map the
# `Compartment` column to ecoinvent main compartments
# using the main_comp dictionary

main_comp = {"Air": "air", "Raw": "natural resource", "Soil": "soil", "Water": "water"}
cfs["ecoinvent_main_compartment"] = cfs["Compartment"].map(main_comp)

print(cfs.loc[cfs["ecoinvent_main_compartment"].isna(), "Compartment"].unique())

[]


In [12]:
# create a `ecoinvent sub compartment` column, and map the
# `Sub-compartment` column to ecoinvent main compartments
# using the sub_comp dictionary

sub_comp = {
    "(unspecified)": "unspecified",
    "agricultural": "agricultural",
    "biotic": "biotic",
    "groundwater": "ground-",
    "groundwater, long-term": "ground-, long-term",
    "high. pop.": "urban air close to ground",
    "in air": "in air",
    "in ground": "in ground",
    "in water": "in water",
    "indoor": "indoor",
    "industrial": "industrial",
    "lake": "surface water",
    "land": "land",
    "low. pop.": "non-urban air or from high stacks",
    "low. pop., long-term": "low population density, long-term",
    "ocean": "ocean",
    "river": "surface water",
    "stratosphere + troposphere": "lower stratosphere + upper troposphere"
}

cfs["ecoinvent_sub_compartment"] = cfs["Sub-compartment"].map(sub_comp)
print(cfs.loc[cfs["ecoinvent_sub_compartment"].isna(), "Sub-compartment"].unique())

[]


In [13]:
cfs.head()

,Unnamed: 0,Impact category,CF unit,Compartment,Sub-compartment,Elem flow name,CAS number,CF value,Elem flow unit,MP or Damage,Native geographical resolution scale,location,ecoinvent_name,ecoinvent_main_compartment,ecoinvent_sub_compartment
6518,6518,Freshwater acidification,kg SO2 eq,Air,(unspecified),Ammonia,007664-41-7,0.180141,kg,Midpoint,Country,AD,Ammonia,air,unspecified
6519,6519,Freshwater acidification,kg SO2 eq,Air,high. pop.,Ammonia,007664-41-7,0.180141,kg,Midpoint,Country,AD,Ammonia,air,urban air close to ground
6520,6520,Freshwater acidification,kg SO2 eq,Air,low. pop.,Ammonia,007664-41-7,0.180141,kg,Midpoint,Country,AD,Ammonia,air,non-urban air or from high stacks
6521,6521,Freshwater acidification,kg SO2 eq,Air,stratosphere + troposphere,Ammonia,007664-41-7,0.180141,kg,Midpoint,Country,AD,Ammonia,air,lower stratosphere + upper troposphere
6522,6522,Freshwater acidification,kg SO2 eq,Air,indoor,Ammonia,007664-41-7,0.180141,kg,Midpoint,Country,AD,Ammonia,air,indoor


In [17]:
cfs_data = {}
for c, category in enumerate(cfs["Impact category"].unique()):
    for t, impact_type in enumerate(cfs["MP or Damage"].unique()):
        subset = cfs.loc[
            (cfs["Impact category"] == category)
            & (cfs["MP or Damage"] == impact_type)
        ]

        cfs_data[f"{category}_{impact_type.lower()}"] = []

        # iterate through rows
        for _, row in subset.iterrows():
            if row["CF value"] != 0:
                data = {
                    "supplier": {
                        "name": row["ecoinvent_name"],
                        "categories": [row["ecoinvent_main_compartment"], row["ecoinvent_sub_compartment"]] if row["ecoinvent_sub_compartment"] != "unspecified" else [row["ecoinvent_main_compartment"]],
                        "matrix": "biosphere"
                    },
                    "consumer": {
                        "location": row["location"],
                        "matrix": "technosphere"
                    },
                    "value": row["CF value"]
                }
                if data not in cfs_data[f"{category}_{impact_type.lower()}"]:
                    cfs_data[f"{category}_{impact_type.lower()}"].append(data)

In [18]:
# save each category as a separate json file
import json

for category in cfs_data:
    with open(f"/Users/romain/GitHub/edges/edges/data/ImpactWorld+ 2.1_{category}.json", "w") as f:
        print(f"Saving {category}")
        json.dump(
            cfs_data[category],
            f,
            indent=2
        )

Saving Freshwater acidification_midpoint
Saving Freshwater acidification_damage
Saving Freshwater eutrophication_midpoint
Saving Freshwater eutrophication_damage
Saving Land occupation, biodiversity_midpoint
Saving Land occupation, biodiversity_damage
Saving Land transformation, biodiversity_midpoint
Saving Land transformation, biodiversity_damage
Saving Marine eutrophication_midpoint
Saving Marine eutrophication_damage
Saving Terrestrial acidification_midpoint
Saving Terrestrial acidification_damage
Saving Water scarcity_midpoint
Saving Water scarcity_damage
Saving Water availability, human health_midpoint
Saving Water availability, human health_damage
Saving Thermally polluted water_midpoint
Saving Thermally polluted water_damage
Saving Water availability, freshwater ecosystem_midpoint
Saving Water availability, freshwater ecosystem_damage
Saving Water availability, terrestrial ecosystem_midpoint
Saving Water availability, terrestrial ecosystem_damage


In [19]:
import json

#load json
with open("/Users/romain/Github/edges/edges/data/ImpactWorld+ 2.1_Freshwater acidification_midpoint.json") as f:
    data = json.load(f)

def get_key(record):
    supplier = record["supplier"]
    consumer = record["consumer"]
    # If order doesn't matter in categories, sort them:
    categories = tuple(sorted(supplier["categories"]))
    return (
        supplier["name"],
        categories,
        supplier["matrix"],
        consumer["location"],
        consumer["matrix"]
    )

seen_keys = set()
duplicates = []
for rec in data:
    key = get_key(rec)
    if key in seen_keys:
        duplicates.append(rec)
    else:
        seen_keys.add(key)

print("Duplicates found:")
for dup in duplicates:
    print(dup)

Duplicates found:
